# Hello, FABRIC: Create your first FABRIC slice

## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

#### Configure the Experiment Parameters



In [ ]:
slice_name_base = 'MySlice_Components_Test'
#test_sites = [ 'UKY', 'RENC', 'LBNL' ]
#test_sites = [ 'TACC', 'MAX', 'UTAH', 'NCSA', 'MICH', 'WASH', 'DALL', 'SALT', 'STAR']
#test_sites = [ 'UTAH', 'NCSA', 'WASH', 'DALL', 'SALT', 'STAR']


#test_sites =[ 'MAX', 'UTAH', 'WASH', 'DALL', 'SALT', 'STAR', 'NCSA']

#test_sites = [ 'DALL', 'TACC', 'NCSA',  'MAX'] 
test_sites = [   'UTAH']


#test_sites = [  'DALL' ]

node_name_base = 'node'


In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import the FABRIC API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Create the Experiment Slice

Load aggragate ads.

In [ ]:
import glob
import json

#aggregate_json_dir=os.environ['HOME']+"/work/git/jupyter-examples/fabric_examples/testing_and_debugging/aggregates/"
aggregate_json_dir="./aggregates/"

aggregates = {}
aggregate_files = glob.glob(aggregate_json_dir+"*.json")
for file_name in aggregate_files:
    print(f"Reading {file_name}")
    site_name = file_name.replace(aggregate_json_dir,"",1).split(".")[0]
    #print(f"{site_name}")
    with open(file_name, 'r') as infile:
        site_info = json.load(infile)
    #print(f"{site_json}")
    aggregates[site_name] = site_info
    

### Create Slice

<img src="./figs/SingleNode.png" width="20%"><br>

Create a single node with basic compute capabilities. The submit function will block until the node is ready and will display a progress bar.


In [ ]:
import time

try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"
        print(f"Creating slice {slice_name}")
        slice = fablib.new_slice(slice_name)
        site_info = aggregates[site]
        
        networks = {}
        
        #create a copy for inner loop
        outer_loop_site_info = site_info['Nodes'].copy()
        inner_loop_site_info = site_info['Nodes'].copy()

        #Add dumbell in each worker
        for worker1_info in outer_loop_site_info:
            worker1 = worker1_info['Name']
            worker1_short_name = worker1.split(".")[0]
            
            print(f"{worker1}")
            
             # Add node
            node1_name = f"{node_name_base}_{worker1_short_name}_1"
            node1 = slice.add_node(name=node1_name, site=site)
            node1.set_host(worker1)
            

            # Add node
            node2_name = f"{node_name_base}_{worker1_short_name}_2"
            node2 = slice.add_node(name=node2_name, site=site)
            node2.set_host(worker1)

            for worker2_info in inner_loop_site_info:
                worker2 = worker2_info['Name']
                worker2_short_name = worker2.split(".")[0]
                
                
                [iface1] = node1.add_component(model="NIC_Basic", name=f'nic_{worker2_short_name}').get_interfaces()
                [iface2] = node2.add_component(model="NIC_Basic", name=f'nic_{worker2_short_name}').get_interfaces()
                
                
                if not worker2_short_name in networks.keys():
                    networks[worker2_short_name] = []
                
                networks[worker2_short_name].append(iface1)
                networks[worker2_short_name].append(iface2)

        
        
        #Create Networks
        net_num = 1
        for worker_short_name, ifaces in networks.items():
            net = slice.add_l2network(name=f'{site}_net{net_num}', interfaces=ifaces)
            net_num += 1


              
    

        #Submit Slice Request
        slice.submit(wait_timeout=600,wait_interval=60)
        
        #time.sleep(30)
except Exception as e:
    print(f"Slice Failed: {e}")
    traceback.print_exc()


In [ ]:
try:
    for site in test_sites:
    #for site in [ 'STAR' ]:

        slice_name=f"{slice_name_base}_{site}"
        print(f"Processing slice {slice_name}")

        #Get Slice
        slice = fablib.get_slice(slice_name)

        #Wait for ssh to be active
        slice.wait_ssh(progress=True, timeout=600,interval=60)
        #time.sleep(30)

        #Run post boo config
        print("post_boot_config")
        slice.post_boot_config()
except Exception as e:
    print(f"Slice Failed: {e}")
    traceback.print_exc()

In [ ]:
try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"
        
        print(f"Waiting for slice {slice_name} ")
        print(json.dumps(slice.get_interface_map(), indent=4, sort_keys=True))
except:
    print("Error")

In [ ]:
try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"    
        slice = fablib.get_slice(name=slice_name)
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"    
        slice = fablib.get_slice(name=slice_name)
        print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"    
        slice = fablib.get_slice(name=slice_name)
        print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"    
        slice = fablib.get_slice(name=slice_name)
        
        for node in slice.get_nodes():
            print(f"{node}")
            for component in node.get_components():
                #print(f"{component}")
                print(f"{component.list_interfaces()}")
            #print(f"{node.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")
 
        

### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

### Test SSH

In [ ]:
try:
    for site in test_sites:
        slice_name=f"{slice_name_base}_{site}"    slice = fablib.get_slice(name=slice_name)
        
        slice = fablib.get_slice(slice_name)
        for node in slice.get_nodes():
            print(f"Node: {node.get_name()}, Host: {node.get_host()}, SSH Test: {node.test_ssh()}")
except Exception as e:
    print(f"Fail: {e}")

## Delete Slice

Please delete your slicd when you are done with your experiment.

In [ ]:
slice_name=f"{slice_name_base}"
print(f"Getting slice {slice_name} ")
try: 
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")